In [2]:
import numpy as np

In [3]:
class Packet():
    def __init__(self, reward, i, classe,prob): 
        self.i = i         
        self.reward = reward
        self.classe = classe 
        self.prob=prob

In [4]:
import random
import numpy as np

class Knapsack():
    #nb_packets est notre lambda dans l'exercice
    def __init__(self, nb_packets, c, rewards, prob, jobs):
        self.rewards = rewards
        self.prob = prob
        self.c = c
        self.jobs = jobs
        self.nb_packets = nb_packets
    
        # generate n packets 
    def BuildPackets(self, nb_packets):
        AllPackets = []
        for i in range(nb_packets):
            #me donne un job un type de job parmis les 4
            class_ = random.choices(jobs,weights=prob,k=1)
            laClasse = class_[0]-1
            AllPackets.append(Packet(rewards[laClasse], i, class_,prob[laClasse]))
        # sort vector depending on rewards
        AllPackets.sort(key=lambda packet: packet.reward, reverse=True)
        return AllPackets

    #Notre policy

 
    def U_alpha(self):
            sommeProb=0
            alpha=0
            for i in range(len(self.prob)):
                sommeProb += self.prob[i]
                # if channel capacity is reaches 
                if sommeProb <= self.c:
                    alpha+=1
                   
                    
                elif sommeProb > self.c:
                    alpha += (self.c- (sommeProb-self.prob[i])) / self.prob[i]
                    break
                
            return alpha 
    def policy(self,packet_class):
      alpha = self.U_alpha()
      if(packet_class<= np.trunc(alpha)):
        return 1
      elif(packet_class== np.trunc(alpha)+1):
        return alpha - np.trunc(alpha)
      else:
        return 0






    #ne fonctionne pas encore
    def estimationY(self, alpha, packets):
        accept = 0
        reject = 0
        for val in packets:
            #alpha égale à 1 par exemple : alors seulement les packet du job 1 qui ont un reward élevé
            if val.classe ==1:
                accept += 1
            else:
                reject += 1
        return (accept / (accept + reject))
    #ne fonctionne pas encore    
    def RobbinsMonroe(self, stepsize, nb_iter):
        alpha_star = 0
        polyak = 0
        alpha = []
        constant_step = 0
        
        for j in range(nb_iter):
            
            Y = self.estimationY(self,alpha_star,self.gen_packets(self.nb_packets, self.p))
            alpha_star = alpha_star + constant_step * (self.c - Y)
            alpha.append(alpha_star)
            if(self.is_saturation_reaches(alpha_star, Y)):
                break

        return alpha_star, Y


if(__name__ == "__main__"):
    nb_packets=10
    c = 0.34
    jobs=[1, 2, 3, 4]
    prob=[1/3, 1/12,1/4, 1/3]
    rewards=[100,50,10,1] 
    knapsack = Knapsack(nb_packets, c, rewards, prob, jobs)
    alpha = knapsack.U_alpha()
    print(knapsack.BuildPackets(nb_packets))
    for i in range(1,len(jobs)+1):
      print(f"the decision for class {i} is :",knapsack.policy(i))
    print(knapsack.U_alpha())

[<__main__.Packet object at 0x0000020002EEE310>, <__main__.Packet object at 0x0000020002EEED30>, <__main__.Packet object at 0x0000020012364A30>, <__main__.Packet object at 0x0000020012364850>, <__main__.Packet object at 0x0000020012372C40>, <__main__.Packet object at 0x0000020002EE93D0>, <__main__.Packet object at 0x00000200123649A0>, <__main__.Packet object at 0x0000020012353F70>, <__main__.Packet object at 0x0000020012364A60>, <__main__.Packet object at 0x0000020012364970>]
the decision for class 1 is : 1
the decision for class 2 is : 0.08000000000000052
the decision for class 3 is : 0
the decision for class 4 is : 0
1.0800000000000005


In [7]:
alpha

1.0800000000000005

In [20]:


class RobbinsM():
    def __init__(self,epsilon,gamma,alpha,c,packet_class, nbr_packets,proba):
        self.epsilon=epsilon
        self.gamma=gamma
        self.alpha=alpha
        self.c=c
        self.packet_class=packet_class
        self.nbr_packets = nbr_packets
        self.proba=proba

  # def policy(self,alphaN):
    
  #     if(self.packet_class<= np.trunc(alphaN)):
  #       return 1
  #     elif(self.packet_class== np.trunc(alphaN)+1):
  #       return alphaN - np.trunc(alphaN)
  #     else:
  #       return 0

    def policy(self,packet_class,alpha ):
        if(packet_class<= np.trunc(alpha)):
            return 1
        elif(packet_class == (np.trunc(alpha)+1)):
            return alpha - np.trunc(alpha)
        else:
            return 0

    def estimationY(self,alpha):
        accepted=0
        for val in self.packet_class:
            #alpha égale à 1 par exemple : alors seulement les packet du job 1 qui ont un reward élevé
            u_alpha= self.policy(val,alpha)
            accepted+=self.nbr_packets*u_alpha*self.proba[val-1]
        return accepted/ self.nbr_packets

    def iterate(self,alpha,epsilon):
        next_alpha = max(0,min(alpha+epsilon*(self.c-self.estimationY(alpha)),len(self.packet_class)))
        return next_alpha

    def robin(self,itr):
        alpha = self.alpha
        for i in range(itr):
            epsilon = self.epsilon/(np.power((itr+1),self.gamma))
            y = self.estimationY(alpha)
            if i%1000==0:
                print(y)
            alpha = self.iterate(alpha,epsilon)
        return alpha,y





In [23]:
rm=RobbinsM(epsilon=0.5,alpha=0.6,c=0.34,packet_class=[1, 2, 3, 4],nbr_packets = 100000,proba=[1/3, 1/12,1/4, 1/3],gamma=0.7)
rm.robin(1000000)

0.2
0.20146452653467178
0.20291373279812353
0.20434777905423943
0.20576682389039774
0.2071710242350098
0.20856053537487473
0.20993551097234983
0.2112961030823458
0.21264246216914268
0.21397473712302695
0.21529307527675748
0.21659762242186065
0.21788852282475127
0.21916591924268716
0.2204299529395573
0.2216807637014994
0.22291848985236531
0.22414326826901276
0.22535523439644392
0.22655452226278416
0.22774126449410284
0.22891559232908126
0.230077635633526
0.23122752291473006
0.2323653813356822
0.23349133672913333
0.23460551361150855
0.23570803519668013
0.23679902340959116
0.23787859889973814
0.23894688105451817
0.24000398801242526
0.24105003667611882
0.2420851427253515
0.24310942062976065
0.24412298366152763
0.24512594390790476
0.2461184122836108
0.24710049854309443
0.24807231129267565
0.24903395800255354
0.24998554501869155
0.250927177574578
0.2518589598028647
0.2527809947468811
0.2536933843720303
0.2545962295770648
0.25548963020524584
0.2563736850553843
0.25724849189276416
0.2581141474

0.3353310854759746
0.33534334388745646
0.33535557011401596
0.3353677642401556
0.3353799263501565
0.3353920565280779
0.33540415485775893
0.33541622142281813
0.33542825630665507
0.3354402595924498
0.33545223136316393
0.3354641717015421
0.3354760806901106
0.3354879584111798
0.3354998049468433
0.33551162037898
0.3355234047892537
0.3355351582591123
0.33554688086979195
0.33555857270231426
0.33557023383748835
0.3355818643559117
0.33559346433796955
0.33560503386383606
0.3356165730134756
0.3356280818666416
0.3356395605028787
0.3356510090015231
0.3356624274417014
0.33567381590233364
0.33568517446213214
0.33569650319960276
0.3357078021930449
0.3357190715205534
0.33573031126001696
0.3357415214891194
0.3357527022853419
0.3357638537259615
0.3357749758880524
0.33578606884848666
0.33579713268393396
0.33580816747086356
0.33581917328554284
0.3358301502040403
0.3358410983022231
0.3358520176557603
0.3358629083401223
0.3358737704305812
0.33588460400221154
0.33589540912988997
0.3359061858882978
0.3359169343

0.3384685199586096
0.33847254091731976
0.33847655131885157
0.33848055119092246
0.3384845405611784
0.3384885194571916
0.3384924879064629
0.3384964459364204
0.33850039357442097
0.33850433084774867
0.3385082577836167
0.3385121744091662
0.33851608075146733
0.33851997683751933
0.3385238626942499
0.33852773834851696
0.3385316038271072
0.33853545915673733
0.3385393043640541
0.33854313947563364
0.3385469645179827
0.33855077951753854
0.3385545845006693
0.33855837949367296
0.338562164522779
0.3385659396141481
0.33856970479387194
0.3385734600879743
0.33857720552241044
0.33858094112306686
0.3385846669157628
0.3385883829262489
0.3385920891802093
0.3385957857032598
0.3385994725209494
0.3386031496587602
0.3386068171421063
0.33861047499633634
0.33861412324673185
0.33861776191850823
0.33862139103681455
0.3386250106267335
0.33862862071328237
0.33863222132141285
0.3386358124760103
0.33863939420189587
0.33864296652382464
0.33864652946648727
0.3386500830545091
0.33865362731245136
0.3386571622648105
0.33866

(1.067643729194326, 0.3389703080591469)

In [24]:
rm=RobbinsM(epsilon=0.5,alpha=1.6,c=0.34,packet_class=[1, 2, 3, 4],nbr_packets = 100000,proba=[1/3, 1/12,1/4, 1/3],gamma=0.7)
rm.robin(1000000)

0.3833333333333333
0.38321956002946406
0.3831060854417026
0.3829929087857583
0.3828800292794002
0.3827674461424508
0.3826551585967809
0.38254316586630427
0.3824314671769732
0.382320061756771
0.38220894883570894
0.38209812764581874
0.3819875974211499
0.38187735739776174
0.38176740681372007
0.3816577449090906
0.38154837092593546
0.381439284108305
0.3813304837022361
0.3812219689557439
0.38111373911881846
0.3810057934434189
0.3808981311834686
0.38079075159484915
0.3806836539353967
0.38057683746489535
0.38047030144507327
0.38036404513959626
0.38025806781406385
0.3801523687360034
0.3800469471748661
0.37994180240202013
0.37983693369074817
0.37973234031623987
0.37962802155558745
0.3795239766877831
0.3794202049937102
0.3793167057561407
0.3792134782597296
0.37911052179101046
0.37900783563838975
0.3789054190921424
0.3788032714444066
0.3787013919891793
0.3785997800223108
0.3784984348415005
0.3783973557462912
0.3782965420380655
0.3781959930200399
0.3780957079972596
0.37799568627659585
0.37789592716

0.3541023746286105
0.3540653483112129
0.35402841920781036
0.3539915870631636
0.3539548516227049
0.35391821263253237
0.3538816698394131
0.353845222990778
0.3538088718347209
0.3537726161199976
0.35373645559602324
0.35370039001287057
0.35366441912126895
0.35362854267260246
0.35359276041890725
0.3535570721128714
0.3535214775078308
0.3534859763577708
0.35345056841732136
0.35341525344175784
0.3533800311869966
0.3533449014095961
0.3533098638667534
0.35327491831630325
0.3532400645167161
0.3532053022270964
0.3531706312071815
0.3531360512173392
0.35310156201856696
0.353067163372489
0.35303285504135645
0.3529986367880432
0.3529645083760467
0.35293046956948526
0.3528965201330967
0.35286265983223586
0.3528288884328743
0.3527952057015981
0.3527616114056053
0.35272810531270565
0.35269468719131886
0.3526613568104727
0.3526281139398005
0.35259495834954113
0.3525618898105369
0.3525289080942308
0.352496012972667
0.352463204218488
0.35243048160493257
0.35239784490583537
0.3523652938956245
0.35233282834932

0.3445414587416137
0.3445295349701039
0.34451764250490674
0.34450578126382614
0.34449395116488224
0.34448215212631034
0.3444703840665598
0.34445864690429473
0.3444469405583928
0.3444352649479442
0.3444236199922519
0.3444120056108308
0.3444004217234071
0.3443888682499175
0.3443773451105086
0.34436585222553784
0.34435438951557085
0.344342956901382
0.3443315543039534
0.3443201816444753
0.3443088388443446
0.3442975258251642
0.3442862425087427
0.3442749888170946
0.3442637646724393
0.34425256999719983
0.3442414047140034
0.34423026874568
0.34421916201526176
0.3442080844459838
0.34419703596128254
0.34418601648479524
0.34417502594035976
0.34416406425201407
0.3441531313439951
0.34414222714073955
0.3441313515668815
0.3441205045472535
0.34410968600688524
0.3440988958710038
0.3440881340650317
0.3440774005145876
0.3440666951454857
0.34405601788373497
0.3440453686555384
0.3440347473872927
0.3440241540055883
0.34401358843720775
0.344003050609126
0.34399254044850996
0.34398205788271785
0.34397160283929

(1.1175186544294156, 0.3431265627554777)

In [44]:
0.2672566225599751*np.power(10,5)

26725.66225599751